# Retrieve GO terms for gene from the Uniprot database based on the classes

In [7]:
import numpy as np
import pandas as pd
import string
import os
from collections import Counter
from collections import defaultdict

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import re
from bioservices import *
import collections
%pylab inline --no-import-all

Populating the interactive namespace from numpy and matplotlib


In [43]:
train = pd.read_csv('..//..//bases/training_variants')
test = pd.read_csv('..//..//bases/test_variants')

for i in train.index:
    print(train.Gene[i], train.Class[i], train.Variation[i])

FAM58A 1 Truncating Mutations
CBL 2 W802*
CBL 2 Q249E
CBL 3 N454D
CBL 4 L399V
CBL 4 V391I
CBL 5 V430M
CBL 1 Deletion
CBL 4 Y371H
CBL 4 C384R
CBL 4 P395A
CBL 4 K382E
CBL 4 R420Q
CBL 4 C381A
CBL 5 P428L
CBL 4 D390Y
CBL 1 Truncating Mutations
CBL 4 Q367P
CBL 5 M374V
CBL 4 Y371S
CBL 6 H94Y
CBL 4 C396R
CBL 4 G375P
CBL 4 S376F
CBL 4 P417A
CBL 4 H398Y
SHOC2 4 S2G
TERT 4 Y846C
TERT 7 C228T
TERT 4 H412Y
TERT 4 H876Q
TERT 7 Promoter Mutations
TERT 4 P704S
TERT 2 Amplification
TERT 7 C250T
DICER1 4 G1809R
DICER1 4 G1809K
DICER1 1 D1709E
DICER1 1 D1709A
DICER1 4 E1705A
DICER1 4 D1810A
DICER1 1 Truncating Mutations
DICER1 4 E1705K
DICER1 1 Deletion
PTPRT 4 T1365M
PTPRT 1 V648G
PTPRT 4 T844M
PTPRT 1 A707T
PTPRT 4 Promoter Hypermethylation
PTPRT 4 R1343L
PTPRT 1 A209T
PTPRT 1 Y280H
PTPRT 4 D927G
PTPRT 1 N510K
PTPRT 1 F248S
PTPRT 1 L708P
PTPRT 4 V995M
PTPRT 1 Y412F
PTPRT 1 F74S
PTPRT 1 R1040L
PTPRT 1 R453C
PTPRT 4 R1209W
PTPRT 4 A1022E
PTPRT 4 Q984K
PTPRT 1 T605M
PTPRT 1 K218T
PTPRT 4 N1125I
RHEB 7 Y3

TP53 4 L145R
TP53 4 P278R
TP53 1 R337L
TP53 1 G266E
TP53BP1 1 Truncating Mutations
TP53BP1 1 Deletion
SMAD2 4 R321Q
SMAD2 1 P305L
SMAD2 5 F346V
SMAD2 1 V319D
SMAD2 5 W368H
SMAD2 4 Deletion
SMAD2 1 D450E
SMAD2 6 Y406H
SMAD2 1 K420A
SMAD2 1 T413N
SMAD2 1 S276L
SMAD2 1 D300H
SMAD2 6 K375A
SMAD2 6 N387P
SMAD2 1 Y426A
SMAD2 6 R428A
SMAD2 1 D300N
SMAD2 1 R310H
SMAD2 1 D450H
SMAD3 1 Y298A
SMAD3 1 K341A
SMAD3 1 V356R
SMAD3 4 D258N
SMAD3 1 R373H
SMAD3 4 R287A
SMAD3 1 Deletion
SMAD3 1 Y297A
SMAD3 1 Y238A
SMAD3 1 R243A
SMAD3 4 R373Q
SMAD3 1 R292A
SMAD3 4 E239A
SMAD3 4 D408Y
SMAD3 1 Y384A
SMAD3 1 W406A
SMAD3 4 D408H
SMAD3 4 R268A
SMAD3 4 D408E
SMAD3 4 V277D
SMAD3 4 S425C
SMAD3 3 Y237A
SMAD3 4 P336S
SMAD4 4 D537E
SMAD4 6 W509R
SMAD4 1 K507Q
SMAD4 4 D537Y
SMAD4 1 G508S
SMAD4 1 R378A
SMAD4 5 R441P
SMAD4 4 R361C
SMAD4 1 L43V
SMAD4 1 Truncating Mutations
SMAD4 4 R497H
SMAD4 4 R515G
SMAD4 6 W398V
SMAD4 4 G419V
SMAD4 5 R420H
SMAD4 6 Y513A
SMAD4 6 V465M
SMAD4 1 D493A
SMAD4 1 V370D
SMAD4 4 R100T
SMAD4 4 P1

TSC2 1 Deletion
TSC2 4 A889P
TSC2 1 W1610G
TSC2 1 R462C
TSC2 4 R1200W
TSC2 4 L792R
TSC2 1 L493V
TSC2 7 L493P
TSC2 4 C696Y
TSC2 1 L826P
TSC2 4 A614D
TSC2 1 L146R
TSC2 4 R611Q
TSC2 1 S1498N
TSC2 1 F615S
TSC2 1 Y598C
TSC2 4 V705E
TSC2 1 L844R
TSC2 1 L448P
TSC2 1 P1709L
TSC2 1 E75G
EWSR1 2 EWSR1-FEV Fusion
EWSR1 7 EWSR1-YY1 Fusion
EWSR1 7 EWSR1-DDIT3 Fusion
EWSR1 7 EWSR1-ETV1 Fusion
EWSR1 2 Fusions
EWSR1 7 EWSR1-FLI1 Fusion
EWSR1 7 EWSR1-NR4A3 Fusion
EWSR1 2 EWSR1-CREB1 Fusion
EWSR1 2 EWSR1-ETV4 Fusion
EWSR1 2 EWSR1-ATF1 Fusion
EWSR1 7 EWSR1-WT1 Fusion
ACVR1 7 R206H
ACVR1 2 G328E
ACVR1 7 G328V
FOXA1 1 G87R
FOXA1 1 L388M
FOXA1 1 Deletion
FOXA1 2 S453fs*
FOXA1 1 L455M
FOXA1 1 F400I
FOXA1 7 Amplification
FOXA1 1 M253K
MDM2 7 Amplification
ARID1A 1 Truncating Mutations
EZH2 9 Y646N
EZH2 9 Y646F
EZH2 9 Y646H
EZH2 9 Y646C
EZH2 9 Y646S
EZH2 9 A677G
EZH2 7 Overexpression
EZH2 7 Amplification
MDM4 7 Amplification
KLF4 2 K409Q
RXRA 6 S247F
RXRA 6 S247Y
MAP3K1 1 T1481fs
MAP3K1 6 Truncating Mutations


SDHC 4 R133*
VEGFA 7 Amplification
MPL 2 S505N
MPL 2 W515K
MPL 7 W515L
MPL 2 P106L
PMS1 4 Q233*
CARM1 1 S217C
CARM1 1 S217T
CARM1 1 S217A
FOXO1 1 T24A
VHL 4 P86H
VHL 1 D121G
VHL 4 R64P
VHL 1 Y98H
VHL 4 S68W
VHL 6 V84L
VHL 5 L128F
VHL 1 Truncating Mutations
VHL 4 V155A
VHL 1 G114R
VHL 4 H115R
VHL 1 L118P
VHL 4 R200W
VHL 1 Y98N
VHL 4 Q164R
VHL 1 A149P
VHL 4 L188Q
VHL 4 R69C
VHL 1 Y112H
VHL 1 I151S
VHL 4 L158P
VHL 4 N78S
VHL 1 F119S
VHL 4 S65W
VHL 4 Q145H
VHL 4 L158Q
VHL 4 H191D
VHL 1 L188V
VHL 1 L118R
VHL 4 W117C
VHL 4 L63P
VHL 1 E70K
VHL 5 P25L
VHL 4 R167W
VHL 4 R82P
VHL 4 S65L
VHL 4 L158V
VHL 1 S65A
VHL 4 Q96P
VHL 1 Deletion
VHL 4 R167Q
MAP2K4 4 V321M
MAP2K4 4 R154W
MAP2K4 4 R228K
MAP2K4 4 P326L
MAP2K4 1 Truncating Mutations
MAP2K4 4 N234I
MAP2K4 2 R134Q
MAP2K4 4 G85R
MAP2K4 4 R304*
RRAS2 7 Q72L
FLT1 6 Amplification
FLT3 7 FLT3 internal tandem duplications
FLT3 7 D835N
FLT3 7 N676K
FLT3 7 Y599_D600insGLYVDFREYEY
FLT3 7 D835A
FLT3 7 F590G
FLT3 3 T167A
FLT3 3 D324N
FLT3 7 Y591D
FLT3 7 F5

BRCA1 5 A1789S
BRCA1 5 R866C
BRCA1 6 V11A
BRCA1 4 D1739V
BRCA1 1 M1775E
BRCA1 3 D67Y
BRCA1 6 D1344H
BRCA1 3 V271L
BRCA1 1 S1841N
BRCA1 4 F1761S
BRCA1 5 E1586G
BRCA1 1 S1841A
BRCA1 6 I89N
BRCA1 6 L82P
BRCA1 5 P798L
BRCA1 6 E29V
BRCA1 4 M1775K
BRCA1 5 E1060A
BRCA1 6 I90T
BRCA1 6 I15T
BRCA1 1 V1736G
BRCA1 1 L1854P
BRCA1 1 E1836K
BRCA1 1 F1704S
BRCA1 6 P1812S
BRCA1 5 H1402Y
BRCA1 5 N550H
BRCA1 4 Q1756fs
BRCA1 4 G1738R
BRCA1 5 S1497A
BRCA1 5 N1647K
BRCA1 5 Q1785H
BRCA1 4 H41R
BRCA1 5 R1589P
BRCA1 5 L165P
BRCA1 4 C24R
BRCA1 5 L1844R
BRCA1 5 C1767S
BRCA1 6 K65M
BRCA1 5 V1809A
BRCA1 1 Y1703H
BRCA1 4 C27A
BRCA1 4 S1655F
BRCA1 6 N1819Y
BRCA1 5 S1486C
BRCA1 5 P1859R
BRCA1 4 Y1853*
BRCA1 5 E1682K
BRCA1 6 K56N
BRCA1 3 S1651F
BRCA1 1 V1714G
BRCA1 5 D1778N
BRCA1 1 Y1703S
BRCA1 5 A1669S
BRCA1 4 W1837R
BRCA1 1 V1833E
BRCA1 5 P1806A
BRCA1 4 M18T
BRCA1 6 M1411T
BRCA1 5 S1140G
BRCA1 4 Y1853C
BRCA1 4 T1700A
BRCA1 6 T1852S
BRCA1 5 H1805P
BRCA1 5 D1778Y
BRCA1 1 T1685A
BRCA1 5 D1733G
BRCA1 1 Q1811R
BRCA1 4 D1

In [9]:
# only use gene from train data -> contains the classes
all_genes = set(train.Gene)
print(len(all_genes))
print(all_genes)

264
{'FAM58A', 'ARID5B', 'SOX9', 'CBL', 'ELF3', 'IDH2', 'IL7R', 'RRAS2', 'CDK4', 'RYBP', 'U2AF1', 'INPP4B', 'KMT2C', 'TERT', 'SHQ1', 'CTCF', 'MDM2', 'NSD1', 'GNAS', 'NF2', 'FGFR4', 'RB1', 'MAP3K1', 'SMAD4', 'MAPK1', 'ASXL1', 'ERCC3', 'ASXL2', 'RICTOR', 'RNF43', 'ARAF', 'LATS2', 'HLA-A', 'STAT3', 'RAD54L', 'PIK3R1', 'CDKN2A', 'ERCC2', 'MTOR', 'DNMT3A', 'SMAD3', 'TSC2', 'ACVR1', 'FAT1', 'CDH1', 'PIK3CA', 'CEBPA', 'KMT2B', 'KDM5A', 'PMS2', 'POLE', 'PTEN', 'JUN', 'ATM', 'FUBP1', 'RHEB', 'FGFR3', 'JAK2', 'KDM5C', 'MPL', 'MAP2K2', 'ESR1', 'FOXP1', 'H3F3A', 'ARID2', 'VHL', 'ALK', 'CTLA4', 'BTK', 'CIC', 'HNF1A', 'EPAS1', 'MET', 'FANCA', 'FGF4', 'RAC1', 'NPM1', 'ERBB2', 'FBXW7', 'EPCAM', 'PAK1', 'PRDM1', 'AURKA', 'TGFBR1', 'TGFBR2', 'SMARCB1', 'BCOR', 'AKT2', 'RET', 'TET2', 'MSH2', 'ATRX', 'FOXL2', 'TMPRSS2', 'AXIN1', 'SMAD2', 'AKT1', 'KNSTRN', 'APC', 'AR', 'WHSC1L1', 'NOTCH2', 'HRAS', 'RXRA', 'ETV6', 'MEN1', 'PDGFRB', 'PTPRT', 'RAD51C', 'ERCC4', 'CDK6', 'RIT1', 'CASP8', 'SMO', 'HIST1H1C', 'AGO

In [10]:
u = UniProt()

In [11]:
res = u.search("ZAP70_HUMAN")
print(res)

Entry	Entry name	Status	Protein names	Gene names	Organism	Length
P43403	ZAP70_HUMAN	reviewed	Tyrosine-protein kinase ZAP-70 (EC 2.7.10.2) (70 kDa zeta-chain associated protein) (Syk-related tyrosine kinase)	ZAP70 SRK	Homo sapiens (Human)	619



In [12]:
u.debugLevel = "INFO"
u.timeout = 100   # some queries are long and requires much more time; default is 1000 seconds

In [13]:
# just an example of query
a = u.search('SLC16A1+AND+organism:9606', frmt='tab', limit=1,
               columns="entry name")

In [14]:
[s.strip() for s in a.splitlines()]

['Entry name', 'MOT1_HUMAN']

In [15]:
gene_entry_dict = {} # here we will keep the gene_entries together with their classes
class_dict = {}
for gene in all_genes:
    gene_classes = list(train.Class[train.Gene==gene])
    keyword = 'gene:%s+AND+organism:9606' %gene #to query database, with gene and organism 9606 is Homo Sapien (human)
    entry_name_tab = u.search(keyword, frmt='tab', limit=1, columns="entry name") 
    entry_name = [s.strip() for s in entry_name_tab.splitlines()][1] # gets the entry name from uniprot i.e. second position in tab
    gene_entry_dict[gene] = entry_name
    class_dict[entry_name] = gene_classes

In [16]:
gene_entry_dict

{'ABL1': 'ABL1_HUMAN',
 'ACVR1': 'ACVR1_HUMAN',
 'AGO2': 'AGO2_HUMAN',
 'AKT1': 'AKT1_HUMAN',
 'AKT2': 'AKT2_HUMAN',
 'AKT3': 'AKT3_HUMAN',
 'ALK': 'TGFR1_HUMAN',
 'APC': 'APC_HUMAN',
 'AR': 'ANDR_HUMAN',
 'ARAF': 'ARAF_HUMAN',
 'ARID1A': 'ARI1A_HUMAN',
 'ARID1B': 'ARI1B_HUMAN',
 'ARID2': 'ARID2_HUMAN',
 'ARID5B': 'ARI5B_HUMAN',
 'ASXL1': 'ASXL1_HUMAN',
 'ASXL2': 'ASXL2_HUMAN',
 'ATM': 'ATM_HUMAN',
 'ATR': 'ATR_HUMAN',
 'ATRX': 'ATRX_HUMAN',
 'AURKA': 'AURKA_HUMAN',
 'AURKB': 'AURKB_HUMAN',
 'AXIN1': 'AXIN1_HUMAN',
 'AXL': 'UFO_HUMAN',
 'B2M': 'B2MG_HUMAN',
 'BAP1': 'RING2_HUMAN',
 'BARD1': 'BARD1_HUMAN',
 'BCL10': 'BCL10_HUMAN',
 'BCL2': 'BCL2_HUMAN',
 'BCL2L11': 'B2L11_HUMAN',
 'BCOR': 'BCOR_HUMAN',
 'BRAF': 'BRAF_HUMAN',
 'BRCA1': 'BRCA1_HUMAN',
 'BRCA2': 'BRCA2_HUMAN',
 'BRD4': 'BRD4_HUMAN',
 'BRIP1': 'FANCJ_HUMAN',
 'BTK': 'BTK_HUMAN',
 'CARD11': 'CAR11_HUMAN',
 'CARM1': 'CARM1_HUMAN',
 'CASP8': 'CASP8_HUMAN',
 'CBL': 'CBL_HUMAN',
 'CCND1': 'CCND1_HUMAN',
 'CCND2': 'CCND2_HUMAN',


In [17]:
gene_entries = list(gene_entry_dict.values())
len(gene_entries)

264

In [18]:
df = u.get_df(gene_entries)
df

INFO:root:fetching information from uniprot for 262 entries
INFO:root:uniprot.get_df 1/2
INFO:root:uniprot.get_df 2/2
INFO:root:uniprot.get_df 3/2


,Entry,Entry name,Gene names,Gene names (primary ),Gene names (synonym ),Gene names (ordered locus ),Gene names (ORF ),Organism,Organism ID,Protein names,...,Miscellaneous [CC],Keywords,Protein existence,Status,Sequence annotation (Features),Protein families,Version,Comments,Cross-reference (null),Pathway.1
0,P04637,P53_HUMAN,[TP53 P53],TP53,P53,NaN,NaN,Homo sapiens (Human),9606,Cellular tumor antigen p53 (Antigen NY-CO-13) ...,...,NaN,"[3D-structure, Acetylation, Activator, Alterna...",Evidence at protein level,reviewed,NaN,[P53 family],257,"[Alternative products (1), Caution (5), Cofact...",NaN,NaN
1,P46531,NOTC1_HUMAN,[NOTCH1 TAN1],NOTCH1,TAN1,NaN,NaN,Homo sapiens (Human),9606,Neurogenic locus notch homolog protein 1 (Notc...,...,NaN,"[3D-structure, ANK repeat, Activator, Angiogen...",Evidence at protein level,reviewed,NaN,[NOTCH family],211,"[Function (1), Involvement in disease (2), Pos...",NaN,NaN
2,P31749,AKT1_HUMAN,[AKT1 PKB RAC],AKT1,PKB RAC,NaN,NaN,Homo sapiens (Human),9606,RAC-alpha serine/threonine-protein kinase (EC ...,...,NaN,"[3D-structure, ATP-binding, Acetylation, Alter...",Evidence at protein level,reviewed,NaN,"[Protein kinase superfamily, AGC Ser/Thr prote...",216,"[Alternative products (1), Catalytic activity ...",NaN,NaN
3,P42336,PK3CA_HUMAN,[PIK3CA],PIK3CA,NaN,NaN,NaN,Homo sapiens (Human),9606,"Phosphatidylinositol 4,5-bisphosphate 3-kinase...",...,MISCELLANEOUS: The avian sarcoma virus 16 geno...,"[3D-structure, ATP-binding, Angiogenesis, Comp...",Evidence at protein level,reviewed,NaN,[PI3/PI4-kinase family],189,"[Catalytic activity (2), Domain (1), Function ...",NaN,NaN
4,P11362,FGFR1_HUMAN,[FGFR1 BFGFR CEK FGFBR FLG FLT2 HBGFR],FGFR1,BFGFR CEK FGFBR FLG FLT2 HBGFR,NaN,NaN,Homo sapiens (Human),9606,Fibroblast growth factor receptor 1 (FGFR-1) (...,...,NaN,"[3D-structure, ATP-binding, Alternative splici...",Evidence at protein level,reviewed,NaN,"[Protein kinase superfamily, Tyr protein kinas...",233,"[Alternative products (1), Catalytic activity ...",NaN,NaN
5,Q06124,PTN11_HUMAN,[PTPN11 PTP2C SHPTP2],PTPN11,PTP2C SHPTP2,NaN,NaN,Homo sapiens (Human),9606,Tyrosine-protein phosphatase non-receptor type...,...,NaN,"[3D-structure, Acetylation, Alternative splici...",Evidence at protein level,reviewed,NaN,"[Protein-tyrosine phosphatase family, Non-rece...",213,"[Alternative products (1), Catalytic activity ...",NaN,NaN
6,P42771,CDN2A_HUMAN,[CDKN2A CDKN2 MTS1],CDKN2A,CDKN2 MTS1,NaN,NaN,Homo sapiens (Human),9606,Cyclin-dependent kinase inhibitor 2A (Cyclin-d...,...,NaN,"[3D-structure, ANK repeat, Acetylation, Altern...",Evidence at protein level,reviewed,NaN,[CDKN2 cyclin-dependent kinase inhibitor family],197,"[Alternative products (1), Caution (2), Functi...",NaN,NaN
7,P17948,VGFR1_HUMAN,[FLT1 FLT FRT VEGFR1],FLT1,FLT FRT VEGFR1,NaN,NaN,Homo sapiens (Human),9606,Vascular endothelial growth factor receptor 1 ...,...,NaN,"[3D-structure, ATP-binding, Alternative splici...",Evidence at protein level,reviewed,NaN,"[Protein kinase superfamily, Tyr protein kinas...",208,"[Alternative products (1), Catalytic activity ...",NaN,NaN
8,P38936,CDN1A_HUMAN,[CDKN1A CAP20 CDKN1 CIP1 MDA6 PIC1 SDI1 WAF1],CDKN1A,CAP20 CDKN1 CIP1 MDA6 PIC1 SDI1 WAF1,NaN,NaN,Homo sapiens (Human),9606,Cyclin-dependent kinase inhibitor 1 (CDK-inter...,...,NaN,"[3D-structure, Acetylation, Cell cycle, Comple...",Evidence at protein level,reviewed,NaN,[CDI family],199,"[Domain (2), Function (1), Induction (1), Post...",NaN,NaN
9,P40692,MLH1_HUMAN,[MLH1 COCA2],MLH1,COCA2,NaN,NaN,Homo sapiens (Human),9606,DNA mismatch repair protein Mlh1 (MutL protein...,...,NaN,"[3D-structure, ATP-binding, Acetylation, Alter...",Evidence at protein level,reviewed,NaN,[DNA mismatch repair MutL/HexB family],200,"[Alternative products (1), Caution (1), Functi...",NaN,NaN


In [31]:
df_new = df[df['Gene ontology (biological process)'].notnull()] # don't consider genes with no biological process

In [32]:
df_new['Gene ontology (biological process)'] = df_new['Gene ontology (biological process)'].apply(lambda x: x.split('; ')) #split functions based on ;


In [33]:
GO_terms_dict = dict(zip(df_new['Entry name'], df_new['Gene ontology (biological process)']))

In [34]:
GO_terms_dict

{'1A02_HUMAN': ['antibacterial humoral response [GO:0019731]',
  'antigen processing and presentation of endogenous peptide antigen via MHC class I [GO:0019885]',
  'antigen processing and presentation of endogenous peptide antigen via MHC class I via ER pathway, TAP-independent [GO:0002486]',
  'antigen processing and presentation of exogenous peptide antigen via MHC class I, TAP-dependent [GO:0002479]',
  'antigen processing and presentation of exogenous peptide antigen via MHC class I, TAP-independent [GO:0002480]',
  'antigen processing and presentation of peptide antigen via MHC class I [GO:0002474]',
  'defense response to Gram-positive bacterium [GO:0050830]',
  'interferon-gamma-mediated signaling pathway [GO:0060333]',
  'positive regulation of CD8-positive, alpha-beta T cell activation [GO:2001187]',
  'positive regulation of CD8-positive, alpha-beta T cell proliferation [GO:2000566]',
  'positive regulation of interferon-gamma production [GO:0032729]',
  'positive regulation

In [35]:
# Find most common GO terms to use as features
def flatten(l): # taken from https://stackoverflow.com/questions/33900770/most-frequent-values-in-a-dictionary
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, str): #replaced basestring with str for Python3
            for sub in flatten(el):
                yield sub
        else:
            yield el



In [37]:
All_GO_terms = set(list(flatten(GO_terms_dict.values())))
len(All_GO_terms)

2892

In [38]:
# Initialization of the 2056 new features with 0's
for terms in All_GO_terms:
    train[terms] = 0

In [26]:
# looping through all classes and getting terms for each class
'''terms_per_class = defaultdict(list)
for entry, terms in GO_terms_dict.items():
    if entry in class_dict:
        gene_classes = class_dict[entry]
        for gene_class in gene_classes:
            terms_per_class[gene_class].extend(terms)
           
        
terms_per_class'''

'terms_per_class = defaultdict(list)\nfor entry, terms in GO_terms_dict.items():\n    if entry in class_dict:\n        gene_classes = class_dict[entry]\n        for gene_class in gene_classes:\n            terms_per_class[gene_class].extend(terms)\n           \n        \nterms_per_class'

In [27]:
# code if we want most commons
'''counter_dict = {}
for classes in terms_per_class:
    counter_dict[classes] = Counter(terms_per_class[classes]).most_common(50)'''


'counter_dict = {}\nfor classes in terms_per_class:\n    counter_dict[classes] = Counter(terms_per_class[classes]).most_common(50)'

In [39]:
# adds the molecular function GO terms to each gene in train data frame
for i in train.index:
    gene = train.Gene[i]
    gene_entry = gene_entry_dict[gene]
    if gene_entry in GO_terms_dict:
        GO_terms = GO_terms_dict[gene_entry]
        train.loc[i, GO_terms] = 1

train.shape

(3321, 2896)

In [40]:
train

,ID,Gene,Variation,Class,epithelial cell proliferation involved in salivary gland morphogenesis [GO:0060664],negative regulation of B cell proliferation [GO:0030889],activation of cysteine-type endopeptidase activity [GO:0097202],positive regulation of fibroblast migration [GO:0010763],mitotic cell cycle [GO:0000278],"mitochondrial electron transport, succinate to ubiquinone [GO:0006121]",...,peptidyl-serine dephosphorylation [GO:0070262],cytoplasmic sequestering of protein [GO:0051220],cardiac right ventricle morphogenesis [GO:0003215],negative regulation of cell growth [GO:0030308],positive regulation of endodeoxyribonuclease activity [GO:0032079],7-methylguanosine mRNA capping [GO:0006370],venous endothelial cell differentiation [GO:0060843],oxidative phosphorylation [GO:0006119],stress-activated protein kinase signaling cascade [GO:0031098],positive regulation of kidney development [GO:0090184]
0,0,FAM58A,Truncating Mutations,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,CBL,W802*,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,CBL,Q249E,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,CBL,N454D,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,CBL,L399V,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,CBL,V391I,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,CBL,V430M,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7,CBL,Deletion,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,8,CBL,Y371H,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,CBL,C384R,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
# saving the train set together with all features from uniprot
train.to_csv("train_uniprot.csv",index=False)